In [1]:
import numpy as np
from time import time
from collections import Counter
from copy import deepcopy
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [25]:
class Chessboard:
    def __init__(self,n=4,board_state=None): # inicjalizacja
        self.size  = n # do w
        self.board = np.zeros((self.size),dtype=np.int) if board_state is None else board_state  
            
    def getPositions(self):
        return  self.board[:]      
        
    def getNumberOfAttacks(self):
        numberOfAttacks =0
        #numberOfAttacks = self.__horizontalAttacks()
        numberOfAttacks += self.__diagonalAttacks()
        #print(numberOfAttacks)
        return numberOfAttacks
    
    def __CHECK(self):
        counter       = Counter(self.board)
        for value in counter.values():
            if value >1:
                raise Exception('MULTIPLE',' VALUES')

    
    def __horizontalAttacks(self):
        counter       = Counter(self.board) 
        counterValues = list(counter.values())
        for i,value in enumerate(counterValues):
            counterValues[i] = value-1
        return np.sum(counterValues)
    
    def __diagonalAttacks(self): # mozliwe do poprawy !!!
        diagonalAttacks = 0
        for i,h1 in enumerate(self.board):
                for j,h2 in enumerate( self.board):
                    if i != j and np.abs(i-j) ==np.abs(h1-h2)  :
                        diagonalAttacks += 1
                        break
        return diagonalAttacks
        
    def cutOutVector(self,begin,end ):
        return self.board[begin:end+1] # włączamy indeks końca
    
    def injectVector(self,particle,begin,end):
        cutOut            = deepcopy(self.board[begin:end])
        print("inject input:", self.board [:]+1 )
        print("inject vetctor: ", particle[:]+1)
        self.board[begin:end] = particle # włączamy indeks końca
        
    
    def getValueOfIndex(self,index):
        return self.board[index]
    
    def setValueOfIndex(self,value,index):
        self.board[index] = value
    
    def getIndexOfValue(self,value,start,stop):
        board = list(self.board)
        return board.index(value,start,stop)
        
    def __correctOffsprings(self,particle,cutOutVector):
        for implant, cutOutInstance in zip(particle,cutOutVector):
            self.board[self.board == implant] = cutOutInstance
            
        
        
    def mutateAtIndexes(self,firstIndex,secondIndex):
        self.board[firstIndex], self.board[secondIndex] = self.board[secondIndex],self.board[firstIndex]
        self.__CHECK()
        

In [14]:
def printQueensPositions(chessboard: Chessboard):
    """
    wyświetl pozycje hetmanów na szachownicy w czytelny dla człowieka sposób
    """
    board = chessboard.getPositions()
    width = np.size(board)
    #print("{0:{width}}".format(board[:]+1, width=width)) # +1 dla czytelności 
    print("{0}".format(board[:]+1)) # +1 dla czytelności 

In [4]:
def printPopulation(population ):
    print("-------------------------------------------------------------------")
    print("  Subject | Subject code ")
    print("__________________")
    for i, chessboard in enumerate(population):
        print("  {}.  |".format(i+1),end=" ")
        printQueensPositions(chessboard)
    print("__________________")

In [5]:
def printStats(result: Chessboard, BestScore,totalTime):
    """
    wyświetl statystyki jednej iteracji eksperymentu
    """
    
    printQueensPositions(result)
    print("Best Score: {0}".format(BestScore))
    print("Time spent: {0}".format(totalTime), end='\n\n')

In [32]:
class EvolutionAlgorithm:
    
    def __init__(self, chessboardSize =4, populationSize=5 ,crossDiscriminator=0.7,
                 mutationDiscriminator=0.2,maxGeneration=1000, FFMAX=0):
        self.__generation            = 0
        self.__FFMAX                 = FFMAX
        self.__chessboardSize        = chessboardSize
        self.__populationSize        = populationSize
        self.__crossDiscriminator    = crossDiscriminator
        self.__mutationDiscriminator = mutationDiscriminator
        self.__maxGeneration         = maxGeneration
        self.__generateInitialPopulation()
        printPopulation(self.__initialPopulation) # TEST - ewentualnie do usunięcia
    
    def __generateInitialPopulation(self):
        self.__initialPopulation = []
        for i in range(populationSize):
            rng        = np.random.default_rng()
            boardState = rng.permutation(self.__chessboardSize )
            chessboard = Chessboard(n=self.__chessboardSize, board_state=boardState)
            self.__initialPopulation.append(chessboard)
       # print("initial shape ", np.shape(self.__initialPopulation))
            
    def doTheEvolution(self): #main function
        currentPopulation = self.__initialPopulation
        evaluateScroes    = self.__evaluate(currentPopulation,self.__populationSize )
        BestIndex         = self.__getIndexOfBestPopulation(evaluateScroes)
       # print(currentPopulation[BestIndex].getPositions())
        while(self.__isGenerationUnderLimit() and self.__isSolutionNotFound(currentPopulation,BestIndex)):
            self.__newPopulation = self.__selection(currentPopulation)
            self.__crossover()
            self.__mutation()
            evaluateScroes       = self.__evaluate(self.__newPopulation,self.__populationSize )
            BestIndex            = self.__getIndexOfBestPopulation(evaluateScroes)
            currentPopulation    = self.__newPopulation
        #    print(np.shape(currentPopulation))
         #   print(currentPopulation[BestIndex].getPositions())
            self.__increseGenerationCounter()
            print("gen: ", self.__generation)
            printPopulation(self.__newPopulation)
            
        BestScore = self.__evaluate(currentPopulation[BestIndex],1)
        return currentPopulation[BestIndex], BestScore
        
    
    def __evaluate(self,currentPopulation, size): # zwróć liczbę ataków między hetmanami na szachownicy 
        evaluateScroes = np.zeros(size,dtype=np.int8)
        #print(size)
        #print(currentPopulation) # list chessboard object
        if type(currentPopulation) != list:
            currentPopulation = [currentPopulation]
        for i, chessboard in zip(range(size),currentPopulation):
            #print(currentPopulation[i].getPositions())
            evaluateScroes[i] = chessboard.getNumberOfAttacks()
        #print(evaluateScroes)
        return evaluateScroes
    
    def __getIndexOfBestPopulation(self, evaluateScroes):
        
        #print("argmin ",np.argmin(evaluateScroes))
        return np.argmin(evaluateScroes)
    
    def __isGenerationUnderLimit(self):
        return self.__generation < self.__maxGeneration
    
    def __isSolutionNotFound(self,currentPopulation,BestIndex):
        BestScore = self.__evaluate(currentPopulation[BestIndex],1)
        return BestScore > self.__FFMAX
    
    def __selection(self,currentPopulation): # selekcja turniejowa
        newPopulation = []
        for i in range(populationSize):
            firstOpponentIndex,secondOpponentIndex = self.__drawIndexes(self.__populationSize)
            if firstOpponentIndex != secondOpponentIndex:
                firstOpponent  = currentPopulation[firstOpponentIndex]
                secondOpponent = currentPopulation[secondOpponentIndex]
                winner         = firstOpponent if self.__isFirstTheWinner(firstOpponent,secondOpponent ) else secondOpponent
                newPopulation.append(deepcopy(winner))
            else:
                withoutRival = currentPopulation[i]
                newPopulation.append(deepcopy(withoutRival))
        return newPopulation
            
    def __drawIndexes(self, size):
        first  = np.random.randint(low=0, high=size-1)
        second = np.random.randint(low=0, high=size-1)
        return first,second
    
    def __isFirstTheWinner(self,firstOpponent,secondOpponent ):
        firstScore  = self.__evaluate(firstOpponent,1)
        secondScore = self.__evaluate(secondOpponent,1)
        return firstScore <= secondScore
    
    def __crossover(self):
        for index in range(0,self.__populationSize-2,2):
            print("cross index :", index)
            randomState = self.__getRandomValue()
            if randomState <= self.__crossDiscriminator:
                self.__cross(index, index +1)
            
            
        print("aftercross")
        printPopulation(self.__newPopulation)
    def __getRandomValue(self):
        return np.random.rand(1)[0]
    
    def __cross(self, firstIndex, secondIndex):
        firstChessboard                = self.__newPopulation[firstIndex]
        secondChessboard               = self.__newPopulation[secondIndex]
        begin, end                     = self.__setBoundaries()
        firstParticle, secondParticle  = self.__cutOutParticles(firstChessboard,secondChessboard, begin, end)
        end +=1 #inclusive end of vector 
        
        firstChessboard.injectVector(secondParticle,begin,end)
        secondChessboard.injectVector(firstParticle,begin,end)
        
        firstMap, secondMap = self.__createMappingDicts(begin,end, firstChessboard, secondChessboard)
        
        self.__correctOffspring(firstChessboard,firstMap,begin, end )
        self.__correctOffspring(secondChessboard,secondMap,begin, end )
        
    def __createMappingDicts(self,begin,end, firstChessboard, secondChessboard):
        firstMap = {}
        secondMap ={}
        for i in range(begin,end):
            firstValue = firstChessboard.getValueOfIndex(i)
            secondValue = secondChessboard.getValueOfIndex(i)
            firstMap[firstValue] = secondValue
            secondMap[secondValue] = firstValue
            
        firstMap = self.__correctMapping(firstMap)
        secondMap = self.__correctMapping(secondMap)
        return firstMap,secondMap
    
    def __correctMapping(self,mappingList):
        sameValues =[]
        for key,value in mappingList.items():
            for key2, value2 in mappingList.items():
                if key == value2:
                    sameValues.append((key,value,key2)) # pierwsze usuwamy potem pod trzecie ustawiamy drugie
        
        for old,val,new in sameValues:
            del mappingList[old]
            mappingList[new] = val
        return mappingList
    
    def __correctOffspring(self,chessboard,mappingList,begin, end):
        for key, value in mappingList.items():
            print(key,value)
            try:
                #print("index of key: " ,key, " -> ",X.index(key,0,begin))
                index = chessboard.getIndexOfValue(key,0,begin)
                chessboard.setValueOfIndex(value,index)
            except ValueError:
                try:
                    #print("index of key: " ,key, " -> ",X.index(key,end))
                    index = chessboard.getIndexOfValue(key,end,self.__chessboardSize)
                    chessboard.setValueOfIndex(value,index)
                except ValueError:
                    pass
                    #print("niema")
                    #printQueensPositions(chessboard)
        
    
    def __setBoundaries(self):
        begin,end = self.__drawIndexes(self.__chessboardSize)
        while begin == end:
            begin,end = self.__drawIndexes(self.__chessboardSize)
        if end < begin:
            begin,end = end,begin
        return begin,end
    
    def __cutOutParticles(self,firstChessboard,secondChessboard, begin, end):
        firstParticle  = deepcopy(firstChessboard.cutOutVector(begin, end))
        secondParticle = deepcopy(secondChessboard.cutOutVector(begin, end))
        return firstParticle, secondParticle
    
    def __mutation(self):
        for index in range(self.__populationSize):
         #   print("mutate index ", index)
            
            randomState = self.__getRandomValue()
            if randomState <= self.__mutationDiscriminator:
                self.__mutate(index)
        print("aftermutation")
        printPopulation(self.__newPopulation)
                
    def __mutate(self,index):
        #print(np.shape(self.__newPopulation))
        chessboard             = self.__newPopulation[index]
        firstIndex,secondIndex = self.__drawIndexes(self.__chessboardSize)
        while firstIndex == secondIndex:
            firstIndex,secondIndex = self.__drawIndexes(self.__chessboardSize)
        chessboard.mutateAtIndexes(firstIndex,secondIndex)
    
    def __increseGenerationCounter(self):
        self.__generation += 1


In [33]:
#MAIN 
crossDiscriminator    = 0.7
mutationDiscriminator = 0.2
populationSize        = 10
boardStart            = 4
boardStop             = 5
MAXGENERATION         = 1000
boardSize             = np.arange(boardStart,boardStop+1)

for size in boardSize:
    print("................................................................................")
    print("Number of Queens: {}".format(size))
    start             = time()
    evolution         = EvolutionAlgorithm(chessboardSize =size, populationSize=populationSize ,
                                            crossDiscriminator=crossDiscriminator,mutationDiscriminator=mutationDiscriminator,
                                           maxGeneration=MAXGENERATION)
    result, BestScore = evolution.doTheEvolution()
    stop              = time()
    totalTime         = stop - start
    printStats(result, BestScore,totalTime )

................................................................................
Number of Queens: 4
-------------------------------------------------------------------
  Subject | Subject code 
__________________
  1.  | [4 1 3 2]
  2.  | [2 1 3 4]
  3.  | [4 3 2 1]
  4.  | [3 1 4 2]
  5.  | [4 2 3 1]
  6.  | [3 2 1 4]
  7.  | [1 2 3 4]
  8.  | [2 4 3 1]
  9.  | [4 3 1 2]
  10.  | [1 3 2 4]
__________________
[3 1 4 2]
Best Score: [0]
Time spent: 0.012002944946289062

................................................................................
Number of Queens: 5
-------------------------------------------------------------------
  Subject | Subject code 
__________________
  1.  | [2 4 5 3 1]
  2.  | [2 3 4 5 1]
  3.  | [3 2 1 5 4]
  4.  | [4 1 2 5 3]
  5.  | [3 5 2 1 4]
  6.  | [5 2 3 1 4]
  7.  | [5 2 1 3 4]
  8.  | [3 1 2 4 5]
  9.  | [5 2 1 4 3]
  10.  | [1 2 3 5 4]
__________________
cross index : 0
inject input: [3 5 2 1 4]
inject vetctor:  [4 1 2 5]
inject input: [4 1 2 5 

In [ ]:
#Nalezy wykonac wykres zmiennosci wartosci funkcji przystosowania
#najlepszego osobnika w generacjach oraz sredniej wartosci funkcji przystosowania z danej
#populacji tez w generacjach (os X - generacje, os Y - wartosc funkcji przystosowania).

In [ ]:
test = np.array([0,1,2,3,4,5,6,7,8,9])
begin = 1 
end = 3
inject = np.arange(begin,end+1)
inject = inject[:] *2
cutOut = deepcopy(test[begin:end+1])
print("BEFORE", test)
print("cut out: ",cutOut)
print("inject  :", inject)


same =[]
for i,value in enumerate( inject):
    if value in cutOut:
        same.append(value)
        
powtorka ={}
for i,value in enumerate(inject):
    if value in joinedTail and cutOut[i] not in same:
        powtorka[i] = value
print(powtorka)


        

print(same)

for index, value in powtorka.items():
    print(index, value)
    test[test == value] = cutOut[index]
test[begin:end+1] = inject
print("AFTER", test)



In [ ]:
X = deepcopy([7,5,1,3,8,2,6,4])
Y = deepcopy([8,3,5,4,6,1,7,2])

#podmianka
print("X: ",X, "\nY: ",Y)
Xcut = [1,3,8]
Ycut = [5,4,6]

begin = 2
end = 5

X[begin:end] = Ycut
Y[begin: end] = Xcut
print("X: ",X, "\nY: ",Y)


#mapping
mapingX = {}
mapingY ={}
for i in range(begin,end):
    mapingX[X[i]] = Y[i]
    mapingY[Y[i]] = X[i]
print("mapping X: ",mapingX)
print("mapping Y: ",mapingY)


#Xmaping
same =[]
for key,value in mapingX.items():
    for key2, value2 in mapingX.items():
        if key == value2:
            same.append((key,value,key2)) # pierwsze usuwamy  pod trzecie ustawiamy drugie
for old,val,new in same:
    del mapingX[old]
    mapingX[new] = val
#ymapping
same =[]
for key,value in mapingY.items():
    for key2, value2 in mapingY.items():
        if key == value2:
            same.append((key,value,key2)) # pierwsze usuwamy  pod trzecie ustawiamy drugie
for old,val,new in same:
    del mapingY[old]
    mapingY[new] = val
    
print("\n")    
print("mapping X: ",mapingX)
print("mapping Y: ",mapingY)

for key, value in mapingX.items():
    print(key,value)
    try:
        print("index of key: " ,key, " -> ",X.index(key,0,begin))
        index = X.index(key,0,begin)
        X[index] = value
    except ValueError:
        try:
            print("index of key: " ,key, " -> ",X.index(key,end))
            index = X.index(key,end)
            X[index] = value
        except ValueError:
            print("niema")
            
print("X after: ", X)


for key, value in mapingY.items():
    print(key,value)
    try:
        print("index of key: " ,key, " -> ",Y.index(key,0,begin))
        index = Y.index(key,0,begin)
        Y[index] = value
    except ValueError:
        try:
            print("index of key: " ,key, " -> ",Y.index(key,end))
            index = Y.index(key,end)
            Y[index] = value
        except ValueError:
            print("niema")
print("Y after: ", Y)

In [ ]:
X = deepcopy([1,2,3,4,5,6,7])
Y = deepcopy([5,4,6,7,2,1,3])

#podmianka
print("X: ",X, "\nY: ",Y)
Xcut = [3,4,5,6]
Ycut = [6,7,2,1]

begin = 2
end = 6

X[begin:end] = Ycut
Y[begin: end] = Xcut
print("X: ",X, "\nY: ",Y)


#mapping
mapingX = {}
mapingY ={}
for i in range(begin,end):
    mapingX[X[i]] = Y[i]
    mapingY[Y[i]] = X[i]
#Xmaping
same =[]
for key,value in mapingX.items():
    for key2, value2 in mapingX.items():
        if key == value2:
            same.append((key,value,key2)) # pierwsze usuwamy  pod trzecie ustawiamy drugie
for old,val,new in same:
    del mapingX[old]
    mapingX[new] = val
#ymapping
same =[]
for key,value in mapingY.items():
    for key2, value2 in mapingY.items():
        if key == value2:
            same.append((key,value,key2)) # pierwsze usuwamy  pod trzecie ustawiamy drugie
for old,val,new in same:
    del mapingY[old]
    mapingY[new] = val
    
print("\n")    
print("mapping X: ",mapingX)
print("mapping Y: ",mapingY)

for key, value in mapingX.items():
    print(key,value)
    try:
        print("index of key: " ,key, " -> ",X.index(key,0,begin))
        index = X.index(key,0,begin)
        X[index] = value
    except ValueError:
        try:
            print("index of key: " ,key, " -> ",X.index(key,end))
            index = X.index(key,end)
            X[index] = value
        except ValueError:
            print("niema")
            
print("X after: ", X)


for key, value in mapingY.items():
    print(key,value)
    try:
        print("index of key: " ,key, " -> ",Y.index(key,0,begin))
        index = Y.index(key,0,begin)
        Y[index] = value
    except ValueError:
        try:
            print("index of key: " ,key, " -> ",Y.index(key,end))
            index = Y.index(key,end)
            Y[index] = value
        except ValueError:
            print("niema")
print("Y after: ", Y)

In [7]:
#TEST czy algorytm działa
from itertools import cycle
X = deepcopy([3, 0, 7, 5, 4, 1, 2, 6])
Y = deepcopy([7, 3, 5, 2, 4, 1, 6, 0])

#podmianka
print("X: ",X, "Y: ",Y)
Xcut = [0, 7, 5, 4, 1, 2]
Ycut = [3, 5, 2, 4, 1, 6]

begin = 1
end = 7

X[begin:end] = Ycut
Y[begin: end] = Xcut
print("X: ",X, "Y: ",Y)


#mapping
mapingX = {}
for i in range(begin,end):
    if X[i] != Y[i]:
        mapingX[X[i]] = Y[i]
print("mapping X: ",mapingX)

#Xmaping
same =[]
temp = iter(mapingX)
for key in temp:
    key2 = next(temp, None)
    value = mapingX[key]
    value2 = mapingX[key2]
    print(key,key2)
    if key == value2 and key != key2 :
        same.append((key,value,key2)) # pierwsze usuwamy  pod trzecie ustawiamy drugie

if type(same) != list:
    same = [same]
print(same)
for old,val,new in same:
    del mapingX[old]
for old,val,new in same:
    mapingX[new] = val   
    

    
 
    
    
    
print("\n")    
print("mapping X: ",mapingX)

for key, value in mapingX.items():
    #print(key,value)
    try:
        print("index of key: " ,key, " -> ",X.index(key,0,begin))
        index = X.index(key,0,begin)
        X[index] = value
    except ValueError:
        try:
            print("index of key: " ,key, " -> ",X.index(key,end))
            index = X.index(key,end)
            X[index] = value
        except ValueError:
            print("niema")
    print("X : ", X)
print("X after: ", X)



X:  [3, 0, 7, 5, 4, 1, 2, 6] Y:  [7, 3, 5, 2, 4, 1, 6, 0]
X:  [3, 3, 5, 2, 4, 1, 6, 6] Y:  [7, 0, 7, 5, 4, 1, 2, 0]
mapping X:  {3: 0, 5: 7, 2: 5, 6: 2}
3 5
2 6
[(2, 5, 6)]


mapping X:  {3: 0, 5: 7, 6: 5}
index of key:  3  ->  0
X :  [0, 3, 5, 2, 4, 1, 6, 6]
niema
X :  [0, 3, 5, 2, 4, 1, 6, 6]
index of key:  6  ->  7
X :  [0, 3, 5, 2, 4, 1, 6, 5]
X after:  [0, 3, 5, 2, 4, 1, 6, 5]
